In [1]:
import pandas as pd
import numpy as np

In [ ]:
!pip install aif360
!pip install BlackBoxAuditing
!pip install fairlearn==0.4.6

# Load and Split Data

In [2]:
from aif360.datasets import AdultDataset

/home/sophie/.local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/sophie/.local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/sophie/.local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/sophie/.local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:519: 

In [3]:
ad = AdultDataset(instance_weights_name='fnlwgt', features_to_drop=[])

W0425 15:08:50.609015 140402936649536 standard_dataset.py:101] Missing Data: 3620 rows removed from AdultDataset.


In [4]:
ad_train, ad_test = ad.split(2)

# Unconstrained Model

In [5]:
from sklearn.linear_model import LogisticRegression
from copy import deepcopy
from aif360.metrics import ClassificationMetric

In [6]:
ad_df_train, ad_attrs_train = ad_train.convert_to_dataframe(de_dummy_code=False, sep='=', set_category=True)
ad_df_test, ad_attrs_test = ad_test.convert_to_dataframe(de_dummy_code=False, sep='=', set_category=True)

In [7]:
Y_train = np.array(ad_df_train[['income-per-year']]).reshape(((len(ad_df_train),))).astype(int)
Xs_train = np.array(ad_df_train.drop(columns='income-per-year'))
Y_test = np.array(ad_df_test[['income-per-year']]).reshape(((len(ad_df_test),))).astype(int)
Xs_test = np.array(ad_df_test.drop(columns='income-per-year'))

In [8]:
clf = LogisticRegression(max_iter = 300, solver='liblinear').fit(Xs_train, Y_train)

In [9]:
predicted_labels_train = clf.predict(Xs_train).reshape((len(Y_train), 1))
predicted_labels = clf.predict(Xs_test).reshape((len(Y_test), 1))

In [10]:
ad_pred = deepcopy(ad_test)
ad_pred_train = deepcopy(ad_train)
ad_pred.labels = predicted_labels
ad_pred_train.labels = predicted_labels_train

In [11]:
u = [{'race': 0}]
p = [{'race': 1}]
metrics = ClassificationMetric(ad_test,ad_pred,unprivileged_groups=u, privileged_groups=p)
test_acc = metrics.accuracy()
DI = metrics.disparate_impact()
EO = metrics.average_abs_odds_difference()
print('Original Test Accuracy:', test_acc)
print('Original Demographic Parity ratio:', DI)
print('Original Average Absolute Odds diff:', EO)

Original Test Accuracy: 0.8516964899300383
Original Demographic Parity ratio: 0.5652787213393603
Original Average Absolute Odds diff: 0.049887323350057494


In [12]:
from mia.membership_inference_attacks import black_box_benchmarks

In [13]:
# Membership inference attack

# train shadow model
shadow_train, shadow_test = ad_test.split(2)
shadow_train_df, _ = shadow_train.convert_to_dataframe(de_dummy_code=False, sep='=', set_category=True)
shadow_test_df, _ = shadow_test.convert_to_dataframe(de_dummy_code=False, sep='=', set_category=True)
shadow_Y_train = np.array(shadow_train_df[['income-per-year']]).reshape(((len(shadow_train_df),))).astype(int)
shadow_Xs_train = np.array(shadow_train_df.drop(columns='income-per-year'))
shadow_Y_test = np.array(shadow_test_df[['income-per-year']]).reshape(((len(shadow_test_df),))).astype(int)
shadow_Xs_test = np.array(shadow_test_df.drop(columns='income-per-year'))

true_pred_on_shadow_train = clf.predict(shadow_Xs_train)
true_pred_on_shadow_test = clf.predict(shadow_Xs_test)

shadow_model = LogisticRegression(max_iter = 300, solver='liblinear').fit(shadow_Xs_train, true_pred_on_shadow_train)


In [14]:
# get outputs
shadow_out_train = shadow_model.predict_proba(shadow_Xs_train)
shadow_out_test = shadow_model.predict_proba(shadow_Xs_test)
true_model_train = clf.predict_proba(Xs_train)
true_model_test = clf.predict_proba(Xs_test)

In [15]:
shadow_train_performance = (shadow_out_train, true_pred_on_shadow_train)
shadow_test_performance = (shadow_out_test, true_pred_on_shadow_test)
target_train_performance = (true_model_train, Y_train)
target_test_performance = (true_model_test, Y_test)

In [36]:
shadow_train_df.head()

,age,education-num,race,sex,capital-gain,capital-loss,hours-per-week,workclass=Federal-gov,workclass=Local-gov,workclass=Private,...,native-country=Scotland,native-country=South,native-country=Taiwan,native-country=Thailand,native-country=Trinadad&Tobago,native-country=United-States,native-country=Vietnam,native-country=Yugoslavia,income-per-year,race-sex
24471,23.0,9.0,1.0,0.0,0.0,0.0,33.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,2
24472,40.0,14.0,1.0,0.0,0.0,1876.0,35.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,2
24473,45.0,10.0,1.0,0.0,0.0,0.0,35.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,2
24475,22.0,10.0,1.0,1.0,0.0,0.0,40.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,3
24476,38.0,13.0,1.0,1.0,0.0,0.0,50.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,3


In [39]:
shadow_train_df['race-sex'] = shadow_train_df['race'].astype(str) + '-' + shadow_train_df['sex'].astype(str)
shadow_train_df['race-sex'] = shadow_train_df['race-sex'].astype('category').cat.codes
shadow_test_df['race-sex'] = shadow_test_df['race'].astype(str) + '-' + shadow_test_df['sex'].astype(str)
shadow_test_df['race-sex'] = shadow_test_df['race-sex'].astype('category').cat.codes
ad_df_train['race-sex'] = ad_df_train['race'].astype(str) + '-' + ad_df_train['sex'].astype(str)
ad_df_train['race-sex'] = ad_df_train['race-sex'].astype('category').cat.codes
ad_df_test['race-sex'] = ad_df_test['race'].astype(str) + '-' + ad_df_test['sex'].astype(str)
ad_df_test['race-sex'] = ad_df_test['race-sex'].astype('category').cat.codes

In [40]:
# run MIA
MIA = black_box_benchmarks(shadow_train_performance,shadow_test_performance,
                         target_train_performance,target_test_performance,num_classes=2)

In [41]:
MIA._mem_inf_benchmarks(shadow_train_df['race-sex'].astype(int), shadow_test_df['race-sex'].astype(int), ad_df_train['race-sex'].astype(int), ad_df_test['race-sex'].astype(int))

For membership inference attack via correctness, the attack acc is 0.500, with train acc 0.847 and test acc 0.848
For membership inference attack via confidence, group attr 0, the attack acc is 0.528
For membership inference attack via confidence, group attr 1, the attack acc is 0.513
For membership inference attack via confidence, group attr 2, the attack acc is 0.508
For membership inference attack via confidence, group attr 3, the attack acc is 0.506
For membership inference attack via confidence, the shadow attack acc is 0.510
For membership inference attack via confidence, the attack acc is 0.501
For membership inference attack via entropy, group attr 0, the attack acc is 0.520
For membership inference attack via entropy, group attr 1, the attack acc is 0.513
For membership inference attack via entropy, group attr 2, the attack acc is 0.506
For membership inference attack via entropy, group attr 3, the attack acc is 0.506
For membership inference attack via entropy, the shadow att

# Demographic Parity (Independence)

## Feldman et al. Repair (Preprocessing)

In [13]:
import aif360.algorithms.preprocessing as AIF

In [14]:
repairer = AIF.DisparateImpactRemover(repair_level=1.0, sensitive_attribute='race')
repaired_train = repairer.fit_transform(ad_train)
repaired_test = repairer.fit_transform(ad_test)

In [15]:
repaired_df_train, repaired_attrs_train = repaired_train.convert_to_dataframe(de_dummy_code=False, sep='=', set_category=True)
repaired_df_test, repaired_attrs_test = repaired_test.convert_to_dataframe(de_dummy_code=False, sep='=', set_category=True)

In [16]:
Y_rep_train = np.array(repaired_df_train[['income-per-year']]).reshape(((len(repaired_df_train),)))
Xs_rep_train = np.array(repaired_df_train.drop(columns='income-per-year'))

Y_rep_test = np.array(repaired_df_test[['income-per-year']]).reshape(((len(repaired_df_test),)))
Xs_rep_test = np.array(repaired_df_test.drop(columns='income-per-year'))

In [17]:
clf_rep = LogisticRegression(max_iter = 300, solver='liblinear').fit(Xs_rep_train, Y_rep_train)

In [18]:
predicted_labels_rep = clf_rep.predict(Xs_rep_test).reshape((len(Y_rep_test), 1))

In [19]:
ad_pred_rep = deepcopy(ad_test)
ad_pred_rep.labels = predicted_labels_rep

In [20]:
metrics_rep = ClassificationMetric(ad_test,ad_pred_rep,unprivileged_groups=u, privileged_groups=p)
test_acc_rep = metrics_rep.accuracy()
DI_rep = metrics_rep.disparate_impact()
EO_rep = metrics_rep.average_abs_odds_difference()
print('Repair Test Accuracy:', test_acc_rep)
print('Repair Demographic Parity ratio:', DI_rep)
print('Repair Average Absolute Odds diff:', EO_rep)

Repair Test Accuracy: 0.8511096256954943
Repair Demographic Parity ratio: 0.5748155706381144
Repair Average Absolute Odds diff: 0.04189026077198628


## Kamishima et al. Regularization (Regularization)

In [21]:
!pip install tensorflow

In [22]:
from aif360.algorithms.inprocessing import PrejudiceRemover

In [47]:
PrejRemover = PrejudiceRemover(eta=0.1, sensitive_attr='race', class_attr='income-per-year')

In [48]:
PrejRemover = PrejRemover.fit(ad_train)
kamishima_pred = PrejRemover.predict(ad_test)

In [49]:
metrics_kamishima = ClassificationMetric(ad_test,kamishima_pred,unprivileged_groups=u, privileged_groups=p)
test_acc_prej = metrics_kamishima.accuracy()
DI_prej = metrics_kamishima.disparate_impact()
EO_prej = metrics_kamishima.average_abs_odds_difference()
print('Prej Remover Test Accuracy:', test_acc_prej)
print('Prej Remover Demographic Parity ratio:', DI_prej)
print('Prej Remover Average Absolute Odds diff:', EO_prej)

Prej Remover Test Accuracy: 0.8453421761768007
Prej Remover Demographic Parity ratio: 0.4978724331429025
Prej Remover Average Absolute Odds diff: 0.0756202852512517


## Agarwal et al (Reduction)

In [21]:
from aif360.algorithms.inprocessing.exponentiated_gradient_reduction import ExponentiatedGradientReduction

In [43]:
estimator = LogisticRegression(solver='liblinear')
np.random.seed(0) #need for reproducibility
exp_grad_red_dp = ExponentiatedGradientReduction(estimator=estimator, 
                                              constraints="DemographicParity",
                                              drop_prot_attr=False)

In [44]:
exp_grad_red_dp.fit(ad_train)
exp_grad_red_pred_dp = exp_grad_red_dp.predict(ad_test)

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().


In [46]:
metrics_red_dp = ClassificationMetric(ad_test,exp_grad_red_pred_dp,unprivileged_groups=u, privileged_groups=p)
test_acc_red_dp = metrics_red_dp.accuracy()
DI_red_dp = metrics_red_dp.disparate_impact()
EO_red_dp = metrics_red_dp.average_abs_odds_difference()
print('DP Reduction Test Accuracy:', test_acc_red_dp)
print('DP Reduction Demographic Parity ratio:', DI_red_dp)
print('DP Reduction Average Absolute Odds diff:', EO_red_dp)

DP Reduction Test Accuracy: 0.8320405411045667
DP Reduction Demographic Parity ratio: 0.9469555766712154
DP Reduction Average Absolute Odds diff: 0.0630372657437408


# Equalized Odds (Separation)

## Hardt et al. (Postprocessing)

In [28]:
from aif360.algorithms.postprocessing import EqOddsPostprocessing

In [30]:
postprocessing = EqOddsPostprocessing(u, p)

In [33]:
postprocessing = postprocessing.fit(ad_train, ad_pred_train)
postprocess_pred = postprocessing.predict(ad_pred)

In [35]:
metrics_post = ClassificationMetric(ad_test,postprocess_pred,unprivileged_groups=u, privileged_groups=p)
test_acc_post = metrics_post.accuracy()
DI_post = metrics_post.disparate_impact()
EO_ppost = metrics_post.average_abs_odds_difference()
print('Postprocessing Test Accuracy:', test_acc_post)
print('Postprocessing Demographic Parity ratio:', DI_post)
print('Postprocessing Average Absolute Odds diff:', EO_ppost)

Postprocessing Test Accuracy: 0.8429309786202139
Postprocessing Demographic Parity ratio: 0.7083046158936913
Postprocessing Average Absolute Odds diff: 0.016684602600477202


## Agarwal et al (Reduction)

In [22]:
estimator = LogisticRegression(solver='liblinear')
np.random.seed(0) #need for reproducibility
exp_grad_red = ExponentiatedGradientReduction(estimator=estimator, 
                                              constraints="EqualizedOdds",
                                              drop_prot_attr=False)

In [23]:
exp_grad_red.fit(ad_train)
exp_grad_red_pred = exp_grad_red.predict(ad_test)

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().


In [24]:
metrics_red_eo = ClassificationMetric(ad_test,exp_grad_red_pred,unprivileged_groups=u, privileged_groups=p)
test_acc_red_eo = metrics_red_eo.accuracy()
DI_red_eo = metrics_red_eo.disparate_impact()
EO_red_eo = metrics_red_eo.average_abs_odds_difference()
print('EO Reduction Test Accuracy:', test_acc_red_eo)
print('EO Reduction Demographic Parity ratio:', DI_red_eo)
print('EO Reduction Average Absolute Odds diff:', EO_red_eo)

EO Reduction Test Accuracy: 0.8395849432292414
EO Reduction Demographic Parity ratio: 0.6803765727104913
EO Reduction Average Absolute Odds diff: 0.009446494651760642


In [44]:
# Membership inference attack

# train shadow model
true_pred_on_shadow_train = exp_grad_red.predict(shadow_train)
true_pred_on_shadow_test = exp_grad_red.predict(shadow_test)

shadow_model = LogisticRegression(max_iter = 300, solver='liblinear').fit(shadow_Xs_train, true_pred_on_shadow_train.labels)

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().


In [56]:
X_df_train = pd.DataFrame(ad_train.features, columns=ad_train.feature_names)
X_df_test = pd.DataFrame(ad_test.features, columns=ad_test.feature_names)

In [57]:
# get outputs
shadow_out_train = shadow_model.predict_proba(shadow_Xs_train)
shadow_out_test = shadow_model.predict_proba(shadow_Xs_test)
true_model_train = exp_grad_red.model.predict_proba(X_df_train)
true_model_test = exp_grad_red.model.predict_proba(X_df_test)

In [58]:
true_pred_on_shadow_train = true_pred_on_shadow_train.labels.astype(int).reshape(len(true_pred_on_shadow_train.labels),)
true_pred_on_shadow_test = true_pred_on_shadow_test.labels.astype(int).reshape(len(true_pred_on_shadow_test.labels),)

AttributeError: 'numpy.ndarray' object has no attribute 'labels'

In [59]:
shadow_train_performance = (shadow_out_train, true_pred_on_shadow_train)
shadow_test_performance = (shadow_out_test, true_pred_on_shadow_test)
target_train_performance = (true_model_train, Y_train)
target_test_performance = (true_model_test, Y_test)

In [60]:
# run MIA
MIA = black_box_benchmarks(shadow_train_performance,shadow_test_performance,
                         target_train_performance,target_test_performance,num_classes=2)

In [61]:
MIA._mem_inf_benchmarks(shadow_train_df['race-sex'].astype(int), shadow_test_df['race-sex'].astype(int), ad_df_train['race-sex'].astype(int), ad_df_test['race-sex'].astype(int))

For membership inference attack via correctness, the attack acc is 0.500, with train acc 0.846 and test acc 0.846
For membership inference attack via confidence, group attr 0, the attack acc is 0.523
For membership inference attack via confidence, group attr 1, the attack acc is 0.507
For membership inference attack via confidence, group attr 2, the attack acc is 0.507
For membership inference attack via confidence, group attr 3, the attack acc is 0.506
For membership inference attack via confidence, the shadow attack acc is 0.508
For membership inference attack via confidence, the attack acc is 0.500
For membership inference attack via entropy, group attr 0, the attack acc is 0.520
For membership inference attack via entropy, group attr 1, the attack acc is 0.512
For membership inference attack via entropy, group attr 2, the attack acc is 0.507
For membership inference attack via entropy, group attr 3, the attack acc is 0.506
For membership inference attack via entropy, the shadow att